In [1]:
from vpython import *

<IPython.core.display.Javascript object>

In [ ]:
import socketio
import eventlet

In [ ]:
#create a Socket.IO server
sio = socketio.Server()
app = socketio.WSGIApp(sio)

@sio.event
def connect(sid, environ):
    print('connect ', sid)

@sio.event
def disconnect(sid):
    print('disconnect ', sid)

# @sio.on('my message',namespace='/chat')
# def another_event(sid,data):
# print(str(data))
# sio.emit('my message',{'data':'foobar'})

@sio.on('my message')
def test(sid,json):
    #print(sid)
    print(sid, json)

In [2]:
class CarModel:
    def __init__(self, a_d, b_d, c, d_left, d_right, e, f, g, name:str):
        self.a_d = a_d          #페달 ~ 시트를 맨앞으로 당겼을때의 거리
        self.b_d = b_d          #차량내부 바닥 ~ 시트를 맨아래로 내렸을때의 거리
        self.c = c              #사이드미러 중앙 ~ 시트를 맨앞으로 당겼을때 눈위치(사람머리두께를 약 17 ~ 18cm 라고 가정, 차량 옆면과 수평이 되는 거리측정.)
        self.d_left = d_left    #좌측 사이드미러 중앙 ~ 시트 중앙까지의 거리 (차량 옆면과 수직되는 거리측정.)
        self.d_right = d_right  #우측 사이드미러 중앙 ~ 시트 중앙까지의 거리 (차량 옆면과 수직되는 거리측정.)
        self.e = e              #차량내부 바닥 ~ 대시보드
        self.f = f              #차량내부 바닥 ~ 사이드미러 중앙까지의 높이
        self.g = g              #대시 ~ 천장
        self.name:str = name    #차량 모델명

class PersonalConst:
    def __init__(self, hip_to_eye, default_side):
        self.hip_to_eye = hip_to_eye        #엉덩이 ~ 눈
        self.default_side = default_side    #사용자 설정 사이드미러 각 활용 여부

class Drivepos:
    def __init__(self, a_u, b_u, lr_angle_left, lr_angle_right, ud_angle, model:CarModel):
        self.a_u = a_u                          #사용자가 이동시킨 x값
        self.b_u = b_u                          #사용자가 이동시킨 y값
        self.lr_angle_left = lr_angle_left      #사용자가 설정시킨 좌우 사이드미러 angle (차량 옆면 기준)
        self.lr_angle_right = lr_angle_right    #사용자가 설정시킨 좌우 사이드미러 angle (차량 옆면 기준)
        self.ud_angle = ud_angle                #사용자가 설정시킨 상하 사이드미러 angle (미러의 기울기)
        self.model:CarModel = model             #CarModel 객체
    
    def __iter__(self):
        for key in self.__dict__:
             yield (key, self.__dict__[key]) if key != 'model' else (key, self.__dict__[key].__dict__)

In [3]:
def Simulation(current:Drivepos, change, driver_height):
    thickness = 10 
    side_mirror_2_door = 10
    car_bottom_thickness = 10
    car_ceil_thickness = 10
    shortest_seat_2_padal = current.model.a_d
    lowest_seat_2_car_bottom = current.model.b_d
    right_side_mirror_2_center_of_seat = current.model.d_right
    left_side_mirro_2_center_of_seat = current.model.d_left
    dash_2_car_bottom = current.model.e
    inner_car_height = ( current.model.e + current.model.g )
    
    car_width =  right_side_mirror_2_center_of_seat + left_side_mirro_2_center_of_seat - (2 * side_mirror_2_door)
    car_length = 2 * car_width
    car_seat_size = (car_width / 2)/2
    car_seat_height = thickness
    car_seat_back_thickness = thickness
    car_seat_back_size = car_seat_size
    padal_length = 20
    padal_height = thickness
    padal_width = (car_width / 2)
    dash_length = 2 * padal_length 
    dash_width = (car_width / 2)
    dash_height = thickness
    door_thickness = thickness

    side_mirror_length = 20
    side_mirror_height = 10
    side_mirror_thickness = thickness
    car_bottom_pos_x = 0
    car_bottom_pos_y = 0
    car_bottom_pos_z = 0
    
    car_seat_pos_x = car_bottom_pos_x + ( (car_length / 2) - padal_length - shortest_seat_2_padal - (car_seat_size / 2) ) + current.a_u
    car_seat_pos_y = car_bottom_pos_y + lowest_seat_2_car_bottom + (car_bottom_thickness / 2) - (car_seat_height / 2) + current.b_u
    car_seat_pos_z = car_bottom_pos_z -  ( car_width / 4 )
    car_seat_back_pos_x = car_seat_pos_x - (car_seat_size / 2) + (car_seat_back_thickness / 2)
    car_seat_back_pos_y = car_seat_pos_y + (car_seat_height / 2) + (car_seat_back_size / 2)
    car_seat_back_pos_z = car_seat_pos_z 
    car_ceil_pos_x = car_bottom_pos_x
    car_ceil_pos_y = car_bottom_pos_y +  inner_car_height + (car_bottom_thickness / 2) + (car_ceil_thickness / 2) 
    car_ceil_pos_z = car_bottom_pos_z
    eye_position_x = car_bottom_pos_x + (car_length/2) - padal_length - current.model.c
    eye_position_y = car_seat_pos_y + driver_height
    eye_position_z = car_seat_pos_z 
    padal_postion_x = car_bottom_pos_x + ( (car_length / 2) - (padal_length / 2) )
    padal_postion_y = car_bottom_pos_y +  (padal_height / 2) + (car_bottom_thickness / 2)
    padal_postion_z = car_bottom_pos_z -  ( car_width / 4 )
    dash_position_x = car_bottom_pos_x + ( (car_length / 2) - (dash_length / 2) )
    dash_position_y = car_bottom_pos_y + (car_bottom_thickness / 2) + dash_2_car_bottom + (dash_height / 2)
    dash_position_z = car_bottom_pos_z -  ( car_width / 4 )
    left_door_position_x = car_bottom_pos_x
    left_door_position_y = car_bottom_pos_y + (inner_car_height / 2)
    left_door_position_z = car_bottom_pos_z - ( ( car_width / 2 ) + ( door_thickness / 2 ) )
    right_door_position_x = car_bottom_pos_x
    right_door_position_y = car_bottom_pos_y + (inner_car_height / 2)
    right_door_position_z = car_bottom_pos_z + ( ( car_width / 2 ) + ( door_thickness / 2 ) )
    left_side_mirror_position_x = dash_position_x 
    left_side_mirror_position_y = car_bottom_pos_y + (car_bottom_thickness / 2) + (side_mirror_height / 2) + current.model.f
    left_side_mirror_position_z = dash_position_z - ( (car_width / 4) + door_thickness + side_mirror_2_door + ( side_mirror_thickness / 2 ) )
    right_side_mirror_position_x = dash_position_x
    right_side_mirror_position_y = car_bottom_pos_y + (car_bottom_thickness / 2) + (side_mirror_height / 2) + current.model.f
    right_side_mirror_position_z = dash_position_z + ( 3*(car_width / 4) + door_thickness + side_mirror_2_door + ( side_mirror_thickness / 2 ) )
    
    car_bottom = box(pos = vec(car_bottom_pos_x, car_bottom_pos_y, car_bottom_pos_z), length = car_length, height=car_bottom_thickness, width = car_width )
    car_ceil = box(pos = vec(car_ceil_pos_x, car_ceil_pos_y, car_ceil_pos_z), length = car_length, height = car_ceil_thickness, width = car_width, opacity = 0.5 )
    car_seat = box(pos = vec(car_seat_pos_x, car_seat_pos_y, car_seat_pos_z), length = car_seat_size, height = car_seat_height , width = car_seat_size )
    car_seat_back = box(pos = vec(car_seat_back_pos_x, car_seat_back_pos_y, car_seat_back_pos_z), length = car_seat_back_thickness , height=car_seat_back_size , width= car_seat_back_size)
    eye = box(pos = vec(eye_position_x, eye_position_y, eye_position_z), length = 5, height = 5, width = 5, color = color.red)
    padal = box(pos= vec(padal_postion_x, padal_postion_y, padal_postion_z), length = padal_length, height = padal_height, width = padal_width )
    dash = box(pos= vec(dash_position_x, dash_position_y, dash_position_z), length = dash_length, height = dash_height, width = dash_width  )
    left_door = box(pos = vec(left_door_position_x, left_door_position_y, left_door_position_z), length = car_length, height = inner_car_height ,width = door_thickness, opacity = 0.5)
    right_door = box(pos = vec(right_door_position_x, right_door_position_y, right_door_position_z), length = car_length, height = inner_car_height ,width = door_thickness, opacity = 0.5)
    left_side_mirror = box(pos = vec(left_side_mirror_position_x, left_side_mirror_position_y, left_side_mirror_position_z), length = side_mirror_length, height = side_mirror_height  , width =side_mirror_thickness)
    right_side_mirror = box(pos = vec(right_side_mirror_position_x, right_side_mirror_position_y, right_side_mirror_position_z), length = side_mirror_length, height = side_mirror_height  , width =side_mirror_thickness)
    left_side_mirror.rotate(angle = current.lr_angle_left , axis = vec(0,1,0))
    left_side_mirror.rotate(angle = current.ud_angle , axis = vec(1,0,0))
    right_side_mirror.rotate(angle = current.lr_angle_right, axis = vec(0,1,0))
    right_side_mirror.rotate(angle = current.ud_angle, axis = vec(1,0,0))
    
    v = vec(0,0,0)
    dv = 0.5 # 0.2
    dv2 = 0.05
    dt = 1 # 0.1
    back_and_forth_sum = current.a_u
    up_and_down_sum = current.b_u
    theta1_sum = current.lr_angle_left
    theta2_sum = current.lr_angle_right
    theta3_sum = current.ud_angle
    theta4_sum = current.ud_angle

In [4]:
Avante = CarModel(34, 30, 74, 55, 125, 77, 77, 38,"Avante")
current_Model_setting = Drivepos(1, 2, 0, 0, 0, Avante)
hip_to_eye = 170*0.44
Simulation(current_Model_setting, 0, hip_to_eye)

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

Task exception was never retrieved
future: <Task finished name='Task-5' coro=<wsperiodic() done, defined at d:\Anaconda3\envs\drivepos\lib\site-packages\vpython\with_notebook.py:161> exception=TypeError("float() argument must be a string or a number, not 'NoneType'")>
Traceback (most recent call last):
  File "d:\Anaconda3\envs\drivepos\lib\site-packages\vpython\with_notebook.py", line 171, in wsperiodic
    baseObj.glow.handle_msg(msg)
  File "d:\Anaconda3\envs\drivepos\lib\site-packages\vpython\vpython.py", line 393, in handle_msg
    cvs.handle_event(evt)
  File "d:\Anaconda3\envs\drivepos\lib\site-packages\vpython\vpython.py", line 3126, in handle_event
    evt['pos'] = list_to_vec(pos)
  File "d:\Anaconda3\envs\drivepos\lib\site-packages\vpython\vpython.py", line 177, in list_to_vec
    return vector(L[0], L[1], L[2])
  File "d:\Anaconda3\envs\drivepos\lib\site-packages\vpython\vector.py", line 20, in __init__
    self._x = float(args[0]) # make sure it's a float; could be numpy.f

In [ ]:
#시트
if 'left'  in k:
    car_seat.pos.x -= dv
    car_seat_back.pos.x -=dv
    eye.pos.x -= dv
    back_and_forth_sum += dv
    print(back_and_forth_sum)
if 'right' in k: 
    car_seat.pos.x += dv
    car_seat_back.pos.x +=dv
    eye.pos.x += dv
    back_and_forth_sum  -= dv
    print(back_and_forth_sum)
if 'down' in k: 
    car_seat.pos.y -= dv
    car_seat_back.pos.y -=dv
    eye.pos.y -= dv
    up_and_down_sum -= dv
    print(up_and_down_sum)
if 'up' in k: 
    car_seat.pos.y += dv
    car_seat_back.pos.y +=dv
    eye.pos.y += dv
    up_and_down_sum += dv
    print(up_and_down_sum)

In [ ]:
eventlet.wsgi.server(eventlet.listen(('0.0.0.0',5000)),app)